# ČVUT NI-KOP Úloha 1: Experimentální vyhodnocení algoritmu
Zadání: https://courses.fit.cvut.cz/NI-KOP/homeworks/files/task1.html

Experimentálně srovnejte algoritmy GSAT a probSAT. Určete, který algoritmus dospěje rychleji (v menším počtu iterací) k řešení obtížných instancí 3-SAT v rozsahu 20-75 proměnných. Zdůvodněte použité metody a metriky, popište interpretaci dat.

Uvažujte pevné parametry:

# UTILS

- knihovny
    - python-sat
    - GSAT - from courses KOP gsat2 https://courses.fit.cvut.cz/NI-KOP/download/index.html
        -  documantation: https://courses.fit.cvut.cz/NI-KOP/download/files/doc/manpage2.html 
    - probSAT - https://github.com/adrianopolus/probSAT  
        - cm: ignored - default 0
        - eps:  default = 1.0

In [8]:
from pysat.formula import CNF
import random
import os
import time
import datetime
import subprocess
import tempfile
from dataclasses import dataclass
import subprocess
import tempfile
from dataclasses import dataclass
import os
import pandas as pd
from tqdm import tqdm
import time

In [3]:


MAX_FLIPS = 20000
MAX_TRIES = 50

RANDOM_SEED = 42

# Generate 3-SAT instances


ratio becouse: https://en.wikipedia.org/wiki/Boolean_satisfiability_problem

Selman, Mitchell, and Levesque (1996) give empirical data on the difficulty of randomly generated 3-SAT formulas, depending on their size parameters. Difficulty is measured in number recursive calls made by a DPLL algorithm. They identified a phase transition region from almost-certainly-satisfiable to almost-certainly-unsatisfiable formulas at the clauses-to-variables ratio at about 4.26.[15]

In [102]:

def generate_3sat_instance(n_variables, ratio=4.2):
    m = int(n_variables * ratio)
    cnf = CNF()

    for _ in range(m):
        clause = []
        vars = random.sample(range(1, n_variables + 1), 3)
        for v in vars:
            lit = v if random.random() < 0.5 else -v
            clause.append(lit)
        cnf.append(clause)
    return cnf


In [103]:
# generate instances and save to files


n_values = [20, 40, 60, 75]

data_num_instances_per_n = 200

output_dir = "data/generated_instances"
os.makedirs(output_dir, exist_ok=True)
for inst_var_count in n_values:
    for inst_id in range(data_num_instances_per_n):
        cnf = generate_3sat_instance(inst_var_count)
        filepath = os.path.join(output_dir, f"sat_n{inst_var_count}_inst{inst_id + 1}.cnf")
        cnf.to_file(filepath)

# GSAT Algorithm

In [19]:


# define constatnt 

GSAT_PROB = 0.4

GSAT_EXE_FILEPATH = "bin/gsat2"

# define output data structure with data class
@dataclass
class AlgorithmOutput:
    iteration_count: int
    clause_satisfied: int
    clause_total: int

def run_gsat(cnf_filepath, seed, max_flips, max_tries,timeout_seconds=None, prob=GSAT_PROB):
    """ Run executable linux from bin/gsat2 with given CNF formula."""

    start = time.monotonic()
    try:
    
        # zavoláme gsat2 s parametry (-max_flips)
        cmd = ["./" + GSAT_EXE_FILEPATH,
               cnf_filepath,
               "-r", seed.__str__(), # random generator
               "-i", max_flips.__str__(), # max flips
               "-p", prob.__str__(), #probability
               "-T", max_tries.__str__(), # max tries
               ]
        #print(f"Running GSAT command: {' '.join(cmd)}")
        result = subprocess.run(cmd, capture_output=True, text=True, timeout=timeout_seconds)
        
        equation_result = result.stdout
        output = result.stderr
        
        #parse output by space
        output_parsed = output.split()
        
        iteration_count = output_parsed[0]
        clause_satisfied = output_parsed[2]
        clause_total = output_parsed[3]
        
        # if exit code is not 0, return None
        if result.returncode != 0:
            raise Exception(f"GSAT failed with exit code {result.returncode}")
        
        
        return AlgorithmOutput(
            iteration_count=int(iteration_count),
            clause_satisfied=int(clause_satisfied),
            clause_total=int(clause_total)
        )
    except subprocess.TimeoutExpired:
        runtime = time.monotonic() - start
        print(f"GSAT timed out after {runtime} seconds.")
        return None

In [14]:
# test run_gsat
output = run_gsat("data/test_data/uf20-91/uf20-01.cnf", seed=10, max_flips=300, max_tries=10)

print(output)

Running GSAT command: ./bin/gsat2 data/test_data/uf20-91/uf20-01.cnf -r 10 -i 300 -p 0.4 -T 10
AlgorithmOutput(iteration_count=16, clause_satisfied=91, clause_total=91)


# probSAT Algorithm

In [42]:

PROBSAT_EXE_FILEPATH = "bin/probSAT"

PROBSAT_CM = 0 # set defualt in ./bin/probSAT
PROBSAT_CB = 2.3
#  cmd = ["./probSAT", filename, "cb=2.3", "cm=0", f"cutoff={max_flips}"]

def run_probsat(cnf_filepath, seed, max_flips, max_tries, timeout_seconds=None, cb=PROBSAT_CB):
    """ Run executable linux from bin/probSAT with given CNF formula."""
    time_start = time.monotonic()
    try:
    
        if max_tries is not None:
            cmd = ["./" + PROBSAT_EXE_FILEPATH,
                   "--cb", cb.__str__(),
                   "-m", max_flips.__str__(),
                   "--runs", max_tries.__str__(),
                   cnf_filepath,
                   seed.__str__()
                   ]
        else:
            cmd = ["./" + PROBSAT_EXE_FILEPATH,
                   "--cb", cb.__str__(),
                   "-m", max_flips.__str__(),
                   cnf_filepath,
                   RANDOM_SEED.__str__()
                   ]
            
        #print(f"Running probSAT command: {' '.join(cmd)}")
        result = subprocess.run(cmd, capture_output=True, text=True, timeout=timeout_seconds)
        
        output = result.stdout
        #output = result.stderr
    
        if result.returncode != 10 and result.returncode != 0:
            raise Exception(f"probSAT failed with exit code {result.returncode}")
        
        # region PARSE OUTPUT
        
        #parse output by space
        max_clauses = None
        satisfied_clauses = None
        total_flips = 0
        # get number of clauses find after "c number of clauses"
        if "c number of clauses" in output:
            for line in output.splitlines():
                if line.startswith("c number of clauses"):
                    max_clauses = int(line.split()[5])
                    
        for line in output.splitlines():
            if line.startswith("c numFlips"):
                number = line.split()[3]
                number = int(''.join(filter(str.isdigit, number)))
                # remove all non digit characters
                if number < max_flips:
                    total_flips = number
        
        
        # get if Satifiable find if in output is "s SATISFIABLE"
        if "s SATISFIABLE" in output and result.returncode == 10:
            # get number of flips from output line starting with "c flips"
            satisfied_clauses = max_clauses
        else:
            if result.returncode == 0:
                # parse all lines starting wiht c UNKOKNOWN
                satisfied_clauses = []
                if "c UNKNOWN" in output:
                    for line in output.splitlines():
                        if line.startswith("c UNKNOWN"):
                            total_flips += max_flips
                            line_parts = line.split()
                            number = line_parts[3]
                            # remove all non digit characters
                            num_satisfied = ''.join(filter(str.isdigit, number))
                            satisfied_clauses.append(int(num_satisfied))
                    
                    # find min in num_satisfied
                    min_value = min(satisfied_clauses)
                    satisfied_clauses = max_clauses - min_value
            else:
                raise Exception(f"probSAT failed with exit code {result.returncode}")
        
        if total_flips is None or satisfied_clauses is None or max_clauses is None:
            raise Exception(f"probSAT output parsing failed.")
        # endregion
        
        return AlgorithmOutput(
            iteration_count=total_flips,
            clause_satisfied=satisfied_clauses,
            clause_total=max_clauses
        )
    except subprocess.TimeoutExpired:
        runtime = time.monotonic() - time_start
        print(f"probSAT timed out after {runtime} seconds.")
        return None


In [18]:
# test run_probsat
output = run_probsat("data/test_data/uf20-91/uf20-01.cnf", seed=10, max_flips=300, max_tries=10)
print(output)

Running probSAT command: ./bin/probSAT --cb 2.3 -m 300 --runs 10 data/test_data/uf20-91/uf20-01.cnf 10
AlgorithmOutput(iteration_count=37, clause_satisfied=91, clause_total=91)


In [40]:
#output = run_probsat("data/generated_instances/sat_n60_inst2.cnf", seed=211876, max_flips=10000, max_tries=10)
output = run_probsat("data/test_data/uf20-91/uf20-01.cnf", seed=10, max_flips=3, max_tries=1)
print(output)

Running probSAT command: ./bin/probSAT --cb 2.3 -m 3 --runs 1 data/test_data/uf20-91/uf20-01.cnf 10
AlgorithmOutput(iteration_count=3, clause_satisfied=86, clause_total=91)


# Experiments

- instances
    - 1000 instances acroos
        - 150 for each category
    - categories
        - n = 20, 30, 40, 50, 60, 75 
- algorithms parameters
    - max_tries
        - gsat default: 1
                - try: 30
        - probsat default: 9223372036854775807
    - max_flips
        - gsat default: 300
                - try 500
        - probsat
                - try 500
        - categories n * 100
- runs
    - each instance run:
        - 100 times  
        - 1000 times (cviceni pro jednu instanci)


- na cviceni příklad pro porovnání dvou algoritmů (ale pisou ze je to moc):
    - instancí v sadě: 1000
    - počet běhů na instanci: 1000 
    - max flips: 500 
    - max tries: 1 
    - promennych: 20
    - klauzuli: 91  

## UTILS

In [49]:

def run_experiment(repeat_for_instances, data_num_instances_per_n, alg_max_tries, alg_timeout_seconds):
    n_values = [20, 40, 60, 75]
    output_dir = "data/generated_instances"

    now = datetime.datetime.now()
    timestamp = now.strftime("%Y%m%d_%H%M%S")
    os.makedirs("results", exist_ok=True)
    result_filename = f"results/experiment_results_{timestamp}.csv"

    instances_filenames = []
    for inst_var_count in n_values:
        for inst_id in range(data_num_instances_per_n):
            cnf_filepath = os.path.join(output_dir, f"sat_n{inst_var_count}_inst{inst_id + 1}.cnf")
            instances_filenames.append(cnf_filepath)


    for idx, cnf_filepath in enumerate(tqdm(instances_filenames, desc="Processing instances")):
        print(f"Processing instance file: {cnf_filepath}")

        # extract info from instance filename
        parts = os.path.basename(cnf_filepath).split("_")
        var_count_filename = int(parts[1][1:])  # extract n from 'sat_n
        inst_id = int(parts[2][4:-4])  # extract instance number from 'instX.cnf'

        # extract info from first line of CNF file
        with open(cnf_filepath, 'r') as f:
            for line in f:
                if line.startswith('p cnf'):
                    parts = line.strip().split()
                    file_var_count = int(parts[2])
                    file_clause_count = int(parts[3])
                    break
        if file_var_count != var_count_filename:
            raise ValueError(f"Variable count mismatch in file {cnf_filepath}: filename says {var_count_filename}, file says {file_var_count}")

        inst_var_count = file_var_count
        inst_clause_count = file_clause_count

        # ONE INSTANCE
        for repeat in tqdm(range(repeat_for_instances), desc=f"  Repeats for instance {inst_id}"):

            # ALGORITHM PARAMETERS
            random_seed = random.randint(1, 1000000).__str__()
            max_flips = inst_var_count * 100
            max_tries = alg_max_tries
            filepath = cnf_filepath
            timeout_seconds = alg_timeout_seconds

            # Run GSAT
            print(f"Running GSAT for n={inst_var_count}, instance={inst_id}, repeat={repeat + 1}, seed={random_seed}")

            gsat_output = run_gsat(filepath, seed=random_seed, max_flips=max_flips, max_tries=max_tries, timeout_seconds=timeout_seconds)
            if gsat_output is not None:
                if gsat_output.clause_total != inst_clause_count:
                    raise ValueError(f"Clause count mismatch in GSAT output for file {cnf_filepath}: expected {inst_clause_count}, got {gsat_output.clause_total}")

                row_gsat ={
                    "ALG_algorithm": "GSAT",
                    "ALG_max_flips": max_flips,
                    "ALG_max_tries": max_tries,
                    "ALG_seed": random_seed,
                    "INST_n_variables": inst_var_count,
                    "INST_instance_id": inst_id,
                    "INST_clause_total": inst_clause_count,
                    "OUT_clause_satisfied": gsat_output.clause_satisfied,
                    "OUT_flips_count": gsat_output.iteration_count,
                }
            else:
                # TIMEOUT
                row_gsat ={
                    "ALG_algorithm": "GSAT",
                    "ALG_max_flips": max_flips,
                    "ALG_max_tries": max_tries,
                    "ALG_seed": random_seed,
                    "INST_n_variables": inst_var_count,
                    "INST_instance_id": inst_id,
                    "INST_clause_total": inst_clause_count,
                    "OUT_clause_satisfied": None,
                    "OUT_flips_count": None,
                }
            pd.DataFrame([row_gsat]).to_csv(result_filename, mode="a", header=not os.path.exists(result_filename), index=False)
            # Run probSAT
            print(f"Running probSAT for n={inst_var_count}, instance={inst_id}, repeat={repeat + 1}, seed={random_seed}")
            probsat_output = run_probsat(filepath, seed=random_seed, max_flips=max_flips, max_tries=max_tries,
                                         timeout_seconds=timeout_seconds)
            if probsat_output is not None:
                if probsat_output.clause_total != inst_clause_count:
                    raise ValueError(f"Clause count mismatch in probSAT output for file {cnf_filepath}: expected {inst_clause_count}, got {probsat_output.clause_total}")
                row_probsat = {
                    "ALG_algorithm": "probSAT",
                    "ALG_max_flips": max_flips,
                    "ALG_max_tries": max_tries,
                    "ALG_seed": random_seed,
                    "INST_n_variables": inst_var_count,
                    "INST_instance_id": inst_id,
                    "INST_clause_total": inst_clause_count,
                    "OUT_clause_satisfied": probsat_output.clause_satisfied,
                    "OUT_flips_count": probsat_output.iteration_count,
                }
            else:
                # TIMEOUT
                row_probsat = {
                    "ALG_algorithm": "probSAT",
                    "ALG_max_flips": max_flips,
                    "ALG_max_tries": max_tries,
                    "ALG_seed": random_seed,
                    "INST_n_variables": inst_var_count,
                    "INST_instance_id": inst_id,
                    "INST_clause_total": inst_clause_count,
                    "OUT_clause_satisfied": None,
                    "OUT_flips_count": None,
                }
            pd.DataFrame([row_probsat]).to_csv(result_filename, mode="a", header=not os.path.exists(result_filename), index=False)


## Experiment 1

In [16]:
# run for 10 instances from each n category
import time

results = []
data_num_instances_per_n = 2

repeat_for_instances = 10

n_values = [20, 40, 60, 75]
output_dir = "data/generated_instances"

import datetime
now = datetime.datetime.now()
timestamp = now.strftime("%Y%m%d_%H%M%S")
os.makedirs("results", exist_ok=True)
result_filename = f"results/experiment_results_{timestamp}.csv"

for inst_var_count in n_values:
    for inst_id in tqdm(range(data_num_instances_per_n), desc=f"Processing n={inst_var_count}"):
        # ONE INSTANCE
        cnf_filepath = os.path.join(output_dir, f"sat_n{inst_var_count}_inst{inst_id + 1}.cnf")
        
        for repeat in tqdm(range(repeat_for_instances), desc=f"  Repeats for instance {inst_id + 1}"):
            # generate random seed
            random_seed = random.randint(1, 1000000).__str__()
            max_flips = inst_var_count * 100

            # Run GSAT
            print(f"Running GSAT for n={inst_var_count}, instance={inst_id + 1}, repeat={repeat + 1}, seed={random_seed}")
            max_tries = 30
        
            gsat_output = run_gsat(cnf_filepath, seed=random_seed, max_flips=max_flips, max_tries=max_tries, timeout_seconds=1 * 60)
            if gsat_output is not None:
                row_gsat ={
                    "algorithm": "GSAT",
                    "max_flips": max_flips,
                    "max_tries": max_tries,
                    "random_seed": random_seed,
                    "n_variables": inst_var_count,
                    "instance": inst_id + 1,
                    "flips_count": gsat_output.iteration_count,
                    "clause_satisfied": gsat_output.clause_satisfied,
                    "clause_total": gsat_output.clause_total,
                    "SAT": gsat_output.clause_satisfied == gsat_output.clause_total
                }
                pd.DataFrame([row_gsat]).to_csv(result_filename, mode="a", header=not os.path.exists(result_filename), index=False)
            
            # Run probSAT
            print(f"Running probSAT for n={inst_var_count}, instance={inst_id + 1}, repeat={repeat + 1}, seed={random_seed}")
            probsat_output = run_probsat(cnf_filepath, seed=random_seed, max_flips=max_flips, timeout_seconds=1 * 60)
            if probsat_output is not None:
            
                row_probsat = {
                    "algorithm": "probSAT",
                    "max_flips": max_flips,
                    "max_tries": None,
                    "random_seed": random_seed,
                    "n_variables": inst_var_count,
                    "instance": inst_id + 1,
                    "flips_count": probsat_output.iteration_count,
                    "clause_satisfied": probsat_output.clause_satisfied,
                    "clause_total": probsat_output.clause_total,
                    "SAT": probsat_output.clause_satisfied == probsat_output.clause_total
                }
                pd.DataFrame([row_probsat]).to_csv(result_filename, mode="a", header=not os.path.exists(result_filename), index=False)
            


  Repeats for instance 1:   0%|          | 0/10 [00:00<?, ?it/s]

Running GSAT for n=20, instance=1, repeat=1, seed=215529
Running probSAT for n=20, instance=1, repeat=1, seed=215529
Running GSAT for n=20, instance=1, repeat=2, seed=839900
Running probSAT for n=20, instance=1, repeat=2, seed=839900
Running GSAT for n=20, instance=1, repeat=3, seed=83786
Running probSAT for n=20, instance=1, repeat=3, seed=83786



  Repeats for instance 1:  30%|███       | 3/10 [00:00<00:00, 12.72it/s]

Running GSAT for n=20, instance=1, repeat=4, seed=568239
Running probSAT for n=20, instance=1, repeat=4, seed=568239
Running GSAT for n=20, instance=1, repeat=5, seed=757987
Running probSAT for n=20, instance=1, repeat=5, seed=757987
Running GSAT for n=20, instance=1, repeat=6, seed=446027
Running probSAT for n=20, instance=1, repeat=6, seed=446027



Processing n=20:  50%|█████     | 1/2 [00:00<00:00,  1.36it/s]

...

Running GSAT for n=75, instance=2, repeat=1, seed=779077
Running probSAT for n=75, instance=2, repeat=1, seed=779077
Running GSAT for n=75, instance=2, repeat=2, seed=972496
Running probSAT for n=75, instance=2, repeat=2, seed=972496
Running GSAT for n=75, instance=2, repeat=3, seed=439513
Running probSAT for n=75, instance=2, repeat=3, seed=439513
Running GSAT for n=75, instance=2, repeat=4, seed=845293
Running probSAT for n=75, instance=2, repeat=4, seed=845293
Running GSAT for n=75, instance=2, repeat=5, seed=389280



  Repeats for instance 2:  60%|██████    | 6/10 [00:00<00:00, 21.59it/s]

Running probSAT for n=75, instance=2, repeat=5, seed=389280
Running GSAT for n=75, instance=2, repeat=6, seed=899335
Running probSAT for n=75, instance=2, repeat=6, seed=899335
Running GSAT for n=75, instance=2, repeat=7, seed=849524
Running probSAT for n=75, instance=2, repeat=7, seed=849524
Running GSAT for n=75, instance=2, repeat=8, seed=340832
Running probSAT for n=75, instance=2, repeat=8, seed=340832
Running GSAT for n=75, instance=2, repeat=9, seed=804907
Running probSAT for n=75, instance=2, repeat=9, seed=804907



Processing n=75: 100%|██████████| 2/2 [10:02<00:00, 301.45s/it]

Running GSAT for n=75, instance=2, repeat=10, seed=989096
Running probSAT for n=75, instance=2, repeat=10, seed=989096


```
Processing n=20:   0%|          | 0/2 [00:00<?, ?it/s]
  Repeats for instance 1:   0%|          | 0/10 [00:00<?, ?it/s][A
Running GSAT for n=20, instance=1, repeat=1, seed=215529
Running probSAT for n=20, instance=1, repeat=1, seed=215529
Running GSAT for n=20, instance=1, repeat=2, seed=839900
Running probSAT for n=20, instance=1, repeat=2, seed=839900
Running GSAT for n=20, instance=1, repeat=3, seed=83786
Running probSAT for n=20, instance=1, repeat=3, seed=83786


  Repeats for instance 1:  30%|███       | 3/10 [00:00<00:00, 12.72it/s][A
Running GSAT for n=20, instance=1, repeat=4, seed=568239
Running probSAT for n=20, instance=1, repeat=4, seed=568239
Running GSAT for n=20, instance=1, repeat=5, seed=757987
Running probSAT for n=20, instance=1, repeat=5, seed=757987
Running GSAT for n=20, instance=1, repeat=6, seed=446027
Running probSAT for n=20, instance=1, repeat=6, seed=446027


  Repeats for instance 1:  60%|██████    | 6/10 [00:00<00:00, 10.84it/s][A
  Repeats for instance 1: 100%|██████████| 10/10 [00:00<00:00, 13.66it/s][A
Processing n=20:  50%|█████     | 1/2 [00:00<00:00,  1.36it/s]
Running GSAT for n=20, instance=1, repeat=7, seed=622915
Running probSAT for n=20, instance=1, repeat=7, seed=622915
Running GSAT for n=20, instance=1, repeat=8, seed=657787
Running probSAT for n=20, instance=1, repeat=8, seed=657787
Running GSAT for n=20, instance=1, repeat=9, seed=964961
Running probSAT for n=20, instance=1, repeat=9, seed=964961
Running GSAT for n=20, instance=1, repeat=10, seed=865045
Running probSAT for n=20, instance=1, repeat=10, seed=865045


  Repeats for instance 2:   0%|          | 0/10 [00:00<?, ?it/s][A
Running GSAT for n=20, instance=2, repeat=1, seed=419106


  Repeats for instance 2:  30%|███       | 3/10 [00:00<00:00, 23.59it/s][A
Running probSAT for n=20, instance=2, repeat=1, seed=419106
Running GSAT for n=20, instance=2, repeat=2, seed=505196
Running probSAT for n=20, instance=2, repeat=2, seed=505196
Running GSAT for n=20, instance=2, repeat=3, seed=878901
Running probSAT for n=20, instance=2, repeat=3, seed=878901
Running GSAT for n=20, instance=2, repeat=4, seed=995154
Running probSAT for n=20, instance=2, repeat=4, seed=995154
Running GSAT for n=20, instance=2, repeat=5, seed=105915
Running probSAT for n=20, instance=2, repeat=5, seed=105915
Running GSAT for n=20, instance=2, repeat=6, seed=339604


  Repeats for instance 2:  60%|██████    | 6/10 [00:00<00:00, 23.47it/s][A
  Repeats for instance 2:  90%|█████████ | 9/10 [00:00<00:00, 23.21it/s][A
Running probSAT for n=20, instance=2, repeat=6, seed=339604
Running GSAT for n=20, instance=2, repeat=7, seed=965493
Running probSAT for n=20, instance=2, repeat=7, seed=965493
Running GSAT for n=20, instance=2, repeat=8, seed=171313
Running probSAT for n=20, instance=2, repeat=8, seed=171313
Running GSAT for n=20, instance=2, repeat=9, seed=709776
Running probSAT for n=20, instance=2, repeat=9, seed=709776
Running GSAT for n=20, instance=2, repeat=10, seed=81086
Running probSAT for n=20, instance=2, repeat=10, seed=81086

  Repeats for instance 2: 100%|██████████| 10/10 [00:00<00:00, 23.53it/s]
Processing n=20: 100%|██████████| 2/2 [00:01<00:00,  1.71it/s]
Processing n=40:   0%|          | 0/2 [00:00<?, ?it/s]
  Repeats for instance 1:   0%|          | 0/10 [00:00<?, ?it/s][A
  Repeats for instance 1:  30%|███       | 3/10 [00:00<00:00, 28.18it/s][A
Running GSAT for n=40, instance=1, repeat=1, seed=912869
Running probSAT for n=40, instance=1, repeat=1, seed=912869
Running GSAT for n=40, instance=1, repeat=2, seed=655714
Running probSAT for n=40, instance=1, repeat=2, seed=655714
Running GSAT for n=40, instance=1, repeat=3, seed=354278
Running probSAT for n=40, instance=1, repeat=3, seed=354278
Running GSAT for n=40, instance=1, repeat=4, seed=620025
Running probSAT for n=40, instance=1, repeat=4, seed=620025
Running GSAT for n=40, instance=1, repeat=5, seed=892581
Running probSAT for n=40, instance=1, repeat=5, seed=892581
Running GSAT for n=40, instance=1, repeat=6, seed=195690


  Repeats for instance 1:  60%|██████    | 6/10 [00:00<00:00, 25.53it/s][A
  Repeats for instance 1:  90%|█████████ | 9/10 [00:00<00:00, 26.21it/s][A
Running probSAT for n=40, instance=1, repeat=6, seed=195690
Running GSAT for n=40, instance=1, repeat=7, seed=198646
Running probSAT for n=40, instance=1, repeat=7, seed=198646
Running GSAT for n=40, instance=1, repeat=8, seed=671802
Running probSAT for n=40, instance=1, repeat=8, seed=671802
Running GSAT for n=40, instance=1, repeat=9, seed=25311
Running probSAT for n=40, instance=1, repeat=9, seed=25311
Running GSAT for n=40, instance=1, repeat=10, seed=884390
Running probSAT for n=40, instance=1, repeat=10, seed=884390

  Repeats for instance 1: 100%|██████████| 10/10 [00:00<00:00, 19.01it/s]
Processing n=40:  50%|█████     | 1/2 [00:00<00:00,  1.89it/s]
  Repeats for instance 2:   0%|          | 0/10 [00:00<?, ?it/s][A
  Repeats for instance 2:  20%|██        | 2/10 [00:00<00:00, 14.71it/s][A
Running GSAT for n=40, instance=2, repeat=1, seed=249475
Running probSAT for n=40, instance=2, repeat=1, seed=249475
Running GSAT for n=40, instance=2, repeat=2, seed=153115
Running probSAT for n=40, instance=2, repeat=2, seed=153115
Running GSAT for n=40, instance=2, repeat=3, seed=139335
Running probSAT for n=40, instance=2, repeat=3, seed=139335
Running GSAT for n=40, instance=2, repeat=4, seed=972382
Running probSAT for n=40, instance=2, repeat=4, seed=972382


  Repeats for instance 2:  50%|█████     | 5/10 [00:00<00:00, 19.78it/s][A
  Repeats for instance 2:  70%|███████   | 7/10 [00:00<00:00, 19.70it/s][A
Running GSAT for n=40, instance=2, repeat=5, seed=108205
Running probSAT for n=40, instance=2, repeat=5, seed=108205
Running GSAT for n=40, instance=2, repeat=6, seed=538262
Running probSAT for n=40, instance=2, repeat=6, seed=538262
Running GSAT for n=40, instance=2, repeat=7, seed=534452
Running probSAT for n=40, instance=2, repeat=7, seed=534452
Running GSAT for n=40, instance=2, repeat=8, seed=597023
Running probSAT for n=40, instance=2, repeat=8, seed=597023
Running GSAT for n=40, instance=2, repeat=9, seed=928580


  Repeats for instance 2: 100%|██████████| 10/10 [00:00<00:00, 20.02it/s][A
Processing n=40: 100%|██████████| 2/2 [00:01<00:00,  1.92it/s]

Running probSAT for n=40, instance=2, repeat=9, seed=928580
Running GSAT for n=40, instance=2, repeat=10, seed=160460
Running probSAT for n=40, instance=2, repeat=10, seed=160460

Processing n=60:   0%|          | 0/2 [00:00<?, ?it/s]
  Repeats for instance 1:   0%|          | 0/10 [00:00<?, ?it/s][A
Running GSAT for n=60, instance=1, repeat=1, seed=851431
Running probSAT for n=60, instance=1, repeat=1, seed=851431
Running GSAT for n=60, instance=1, repeat=2, seed=986182
Running probSAT for n=60, instance=1, repeat=2, seed=986182
Running GSAT for n=60, instance=1, repeat=3, seed=234124


  Repeats for instance 1:  30%|███       | 3/10 [00:00<00:00, 20.21it/s][A
Running probSAT for n=60, instance=1, repeat=3, seed=234124
Running GSAT for n=60, instance=1, repeat=4, seed=136346
Running probSAT for n=60, instance=1, repeat=4, seed=136346
Running GSAT for n=60, instance=1, repeat=5, seed=302402


  Repeats for instance 1:  60%|██████    | 6/10 [00:00<00:00, 20.95it/s][A
Running probSAT for n=60, instance=1, repeat=5, seed=302402
Running GSAT for n=60, instance=1, repeat=6, seed=900330
Running probSAT for n=60, instance=1, repeat=6, seed=900330
Running GSAT for n=60, instance=1, repeat=7, seed=744727
Running probSAT for n=60, instance=1, repeat=7, seed=744727
Running GSAT for n=60, instance=1, repeat=8, seed=97502
Running probSAT for n=60, instance=1, repeat=8, seed=97502
Running GSAT for n=60, instance=1, repeat=9, seed=751356
Running probSAT for n=60, instance=1, repeat=9, seed=751356


  Repeats for instance 1: 100%|██████████| 10/10 [00:00<00:00, 20.35it/s][A
Processing n=60:  50%|█████     | 1/2 [00:00<00:00,  2.01it/s]
Running GSAT for n=60, instance=1, repeat=10, seed=66193
Running probSAT for n=60, instance=1, repeat=10, seed=66193


  Repeats for instance 2:   0%|          | 0/10 [00:00<?, ?it/s][A
Running GSAT for n=60, instance=2, repeat=1, seed=119007
Running probSAT for n=60, instance=2, repeat=1, seed=119007


  Repeats for instance 2:  10%|█         | 1/10 [01:00<09:02, 60.26s/it][A
probSAT timed out after 60.03268116200343 seconds.
Running GSAT for n=60, instance=2, repeat=2, seed=361297
Running probSAT for n=60, instance=2, repeat=2, seed=361297


  Repeats for instance 2:  20%|██        | 2/10 [02:00<08:01, 60.21s/it][A
probSAT timed out after 60.00640669999848 seconds.
Running GSAT for n=60, instance=2, repeat=3, seed=68256
Running probSAT for n=60, instance=2, repeat=3, seed=68256


  Repeats for instance 2:  30%|███       | 3/10 [03:00<07:01, 60.20s/it][A
probSAT timed out after 60.0058531790055 seconds.
Running GSAT for n=60, instance=2, repeat=4, seed=153428
Running probSAT for n=60, instance=2, repeat=4, seed=153428


  Repeats for instance 2:  40%|████      | 4/10 [04:00<06:01, 60.21s/it][A
probSAT timed out after 60.00593640800071 seconds.
Running GSAT for n=60, instance=2, repeat=5, seed=816948
Running probSAT for n=60, instance=2, repeat=5, seed=816948


  Repeats for instance 2:  50%|█████     | 5/10 [05:01<05:00, 60.19s/it][A
probSAT timed out after 60.00782194099884 seconds.
Running GSAT for n=60, instance=2, repeat=6, seed=494325
Running probSAT for n=60, instance=2, repeat=6, seed=494325


  Repeats for instance 2:  60%|██████    | 6/10 [06:01<04:00, 60.22s/it][A
probSAT timed out after 60.007073995999235 seconds.
Running GSAT for n=60, instance=2, repeat=7, seed=809157
Running probSAT for n=60, instance=2, repeat=7, seed=809157


  Repeats for instance 2:  70%|███████   | 7/10 [07:01<03:00, 60.22s/it][A
probSAT timed out after 60.00733149100415 seconds.
Running GSAT for n=60, instance=2, repeat=8, seed=963254
Running probSAT for n=60, instance=2, repeat=8, seed=963254


  Repeats for instance 2:  80%|████████  | 8/10 [08:01<02:00, 60.21s/it][A
probSAT timed out after 60.006436878000386 seconds.
Running GSAT for n=60, instance=2, repeat=9, seed=693157
Running probSAT for n=60, instance=2, repeat=9, seed=693157


  Repeats for instance 2:  90%|█████████ | 9/10 [09:01<01:00, 60.22s/it][A
probSAT timed out after 60.00555306200113 seconds.
Running GSAT for n=60, instance=2, repeat=10, seed=79874
Running probSAT for n=60, instance=2, repeat=10, seed=79874


  Repeats for instance 2: 100%|██████████| 10/10 [10:02<00:00, 60.21s/it][A
Processing n=60: 100%|██████████| 2/2 [10:02<00:00, 301.31s/it]

probSAT timed out after 60.006072732001485 seconds.

Processing n=75:   0%|          | 0/2 [00:00<?, ?it/s]
  Repeats for instance 1:   0%|          | 0/10 [00:00<?, ?it/s][A
Running GSAT for n=75, instance=1, repeat=1, seed=763782
Running probSAT for n=75, instance=1, repeat=1, seed=763782


  Repeats for instance 1:  10%|█         | 1/10 [01:00<09:02, 60.26s/it][A
probSAT timed out after 60.00460721000127 seconds.
Running GSAT for n=75, instance=1, repeat=2, seed=514844
Running probSAT for n=75, instance=1, repeat=2, seed=514844


  Repeats for instance 1:  20%|██        | 2/10 [02:00<08:01, 60.25s/it][A
probSAT timed out after 60.006180994001625 seconds.
Running GSAT for n=75, instance=1, repeat=3, seed=652206
Running probSAT for n=75, instance=1, repeat=3, seed=652206


  Repeats for instance 1:  30%|███       | 3/10 [03:00<07:01, 60.24s/it][A
probSAT timed out after 60.00537977500062 seconds.
Running GSAT for n=75, instance=1, repeat=4, seed=389640
Running probSAT for n=75, instance=1, repeat=4, seed=389640


  Repeats for instance 1:  40%|████      | 4/10 [04:00<06:01, 60.24s/it][A
probSAT timed out after 60.00498090200563 seconds.
Running GSAT for n=75, instance=1, repeat=5, seed=802907
Running probSAT for n=75, instance=1, repeat=5, seed=802907


  Repeats for instance 1:  50%|█████     | 5/10 [05:01<05:01, 60.24s/it][A
probSAT timed out after 60.00711036899884 seconds.
Running GSAT for n=75, instance=1, repeat=6, seed=437146
Running probSAT for n=75, instance=1, repeat=6, seed=437146


  Repeats for instance 1:  60%|██████    | 6/10 [06:01<04:00, 60.24s/it][A
probSAT timed out after 60.005923165997956 seconds.
Running GSAT for n=75, instance=1, repeat=7, seed=138367
Running probSAT for n=75, instance=1, repeat=7, seed=138367


  Repeats for instance 1:  70%|███████   | 7/10 [07:01<03:00, 60.24s/it][A
probSAT timed out after 60.005954113999906 seconds.
Running GSAT for n=75, instance=1, repeat=8, seed=39850
Running probSAT for n=75, instance=1, repeat=8, seed=39850


  Repeats for instance 1:  80%|████████  | 8/10 [08:01<02:00, 60.24s/it][A
probSAT timed out after 60.006234173997655 seconds.
Running GSAT for n=75, instance=1, repeat=9, seed=756309
Running probSAT for n=75, instance=1, repeat=9, seed=756309


  Repeats for instance 1:  90%|█████████ | 9/10 [09:02<01:00, 60.24s/it][A
probSAT timed out after 60.006050153002434 seconds.
Running GSAT for n=75, instance=1, repeat=10, seed=403228
Running probSAT for n=75, instance=1, repeat=10, seed=403228


  Repeats for instance 1: 100%|██████████| 10/10 [10:02<00:00, 60.24s/it][A
Processing n=75:  50%|█████     | 1/2 [10:02<10:02, 602.44s/it]
probSAT timed out after 60.00462912199873 seconds.


  Repeats for instance 2:   0%|          | 0/10 [00:00<?, ?it/s][A
  Repeats for instance 2:  30%|███       | 3/10 [00:00<00:00, 20.27it/s][A
Running GSAT for n=75, instance=2, repeat=1, seed=779077
Running probSAT for n=75, instance=2, repeat=1, seed=779077
Running GSAT for n=75, instance=2, repeat=2, seed=972496
Running probSAT for n=75, instance=2, repeat=2, seed=972496
Running GSAT for n=75, instance=2, repeat=3, seed=439513
Running probSAT for n=75, instance=2, repeat=3, seed=439513
Running GSAT for n=75, instance=2, repeat=4, seed=845293
Running probSAT for n=75, instance=2, repeat=4, seed=845293
Running GSAT for n=75, instance=2, repeat=5, seed=389280


  Repeats for instance 2:  60%|██████    | 6/10 [00:00<00:00, 21.59it/s][A
Running probSAT for n=75, instance=2, repeat=5, seed=389280
Running GSAT for n=75, instance=2, repeat=6, seed=899335
Running probSAT for n=75, instance=2, repeat=6, seed=899335
Running GSAT for n=75, instance=2, repeat=7, seed=849524
Running probSAT for n=75, instance=2, repeat=7, seed=849524
Running GSAT for n=75, instance=2, repeat=8, seed=340832
Running probSAT for n=75, instance=2, repeat=8, seed=340832
Running GSAT for n=75, instance=2, repeat=9, seed=804907
Running probSAT for n=75, instance=2, repeat=9, seed=804907


  Repeats for instance 2: 100%|██████████| 10/10 [00:00<00:00, 21.67it/s][A
Processing n=75: 100%|██████████| 2/2 [10:02<00:00, 301.45s/it]
Running GSAT for n=75, instance=2, repeat=10, seed=989096
Running probSAT for n=75, instance=2, repeat=10, seed=989096



```

## Experiment 2

In [17]:
# runnig with set maxrestarts for probSAT

repeat_for_instances = 10
data_num_instances_per_n = 2

# run for 10 instances from each n category
import time

n_values = [20, 40, 60, 75]
output_dir = "data/generated_instances"

import datetime
now = datetime.datetime.now()
timestamp = now.strftime("%Y%m%d_%H%M%S")
os.makedirs("results", exist_ok=True)
result_filename = f"results/experiment_results_{timestamp}.csv"

instances_filenames = []
for inst_var_count in n_values:
    for inst_id in range(data_num_instances_per_n):
        cnf_filepath = os.path.join(output_dir, f"sat_n{inst_var_count}_inst{inst_id + 1}.cnf")
        instances_filenames.append(cnf_filepath)


for idx, cnf_filepath in enumerate(tqdm(instances_filenames, desc="Processing instances")):
    print(f"Processing instance file: {cnf_filepath}")
    
    # extract info from instance filename
    parts = os.path.basename(cnf_filepath).split("_")
    inst_var_count = int(parts[1][1:])  # extract n from 'sat_n
    inst_id = int(parts[2][4:-4])  # extract instance number from 'instX.cnf'
    
     # ONE INSTANCE
    for repeat in tqdm(range(repeat_for_instances), desc=f"  Repeats for instance {inst_id}"):
            
            # ALGORITHM PARAMETERS
            random_seed = random.randint(1, 1000000).__str__()
            max_flips = inst_var_count * 100
            max_tries = 30
            filepath = cnf_filepath
            timeout_seconds = 1 * 60
            
            # Run GSAT
            print(f"Running GSAT for n={inst_var_count}, instance={inst_id}, repeat={repeat + 1}, seed={random_seed}")

            gsat_output = run_gsat(filepath, seed=random_seed, max_flips=max_flips, max_tries=max_tries, timeout_seconds=timeout_seconds)
            if gsat_output is not None:
                row_gsat ={
                    "ALG_algorithm": "GSAT",
                    "ALG_max_flips": max_flips,
                    "ALG_max_tries": max_tries,
                    "ALG_seed": random_seed,
                    "INST_n_variables": inst_var_count,
                    "INST_instance_id": inst_id,
                    "OUT_flips_count": gsat_output.iteration_count,
                    "OUT_clause_satisfied": gsat_output.clause_satisfied,
                    "OUT_clause_total": gsat_output.clause_total,
                    "OUT_satisfiable": gsat_output.clause_satisfied == gsat_output.clause_total
                }
                pd.DataFrame([row_gsat]).to_csv(result_filename, mode="a", header=not os.path.exists(result_filename), index=False)

            # Run probSAT
            print(f"Running probSAT for n={inst_var_count}, instance={inst_id}, repeat={repeat + 1}, seed={random_seed}")
            probsat_output = run_probsat(filepath, seed=random_seed, max_flips=max_flips, max_tries=max_tries,
                                         timeout_seconds=timeout_seconds)
            if probsat_output is not None:
                row_probsat = {
                    "ALG_algorithm": "probSAT",
                    "ALG_max_flips": max_flips,
                    "ALG_max_tries": None,
                    "ALG_seed": random_seed,
                    "INST_n_variables": inst_var_count,
                    "INST_instance_id": inst_id,
                    "OUT_flips_count": probsat_output.iteration_count,
                    "OUT_clause_satisfied": probsat_output.clause_satisfied,
                    "OUT_clause_total": probsat_output.clause_total,
                    "OUT_satisfiable": probsat_output.clause_satisfied == probsat_output.clause_total
                }
                pd.DataFrame([row_probsat]).to_csv(result_filename, mode="a", header=not os.path.exists(result_filename), index=False)
            


  Repeats for instance 1:  10%|█         | 1/10 [00:00<00:01,  8.13it/s]

Running GSAT for n=20, instance=1, repeat=1, seed=429735
Running probSAT for n=20, instance=1, repeat=1, seed=429735
Running GSAT for n=20, instance=1, repeat=2, seed=699668



  Repeats for instance 1:  20%|██        | 2/10 [00:00<00:00,  8.12it/s]

Running probSAT for n=20, instance=1, repeat=2, seed=699668
Running GSAT for n=20, instance=1, repeat=3, seed=765566
Running probSAT for n=20, instance=1, repeat=3, seed=765566
Running GSAT for n=20, instance=1, repeat=4, seed=573272
Running probSAT for n=20, instance=1, repeat=4, seed=573272
Running GSAT for n=20, instance=1, repeat=5, seed=704830



  Repeats for instance 1:  80%|████████  | 8/10 [00:00<00:00, 13.95it/s]

...

Running GSAT for n=75, instance=2, repeat=4, seed=796931
Running probSAT for n=75, instance=2, repeat=4, seed=796931



  Repeats for instance 2:  60%|██████    | 6/10 [00:00<00:00, 19.10it/s]

Running GSAT for n=75, instance=2, repeat=5, seed=591420
Running probSAT for n=75, instance=2, repeat=5, seed=591420
Running GSAT for n=75, instance=2, repeat=6, seed=803010
Running probSAT for n=75, instance=2, repeat=6, seed=803010
Running GSAT for n=75, instance=2, repeat=7, seed=868241
Running probSAT for n=75, instance=2, repeat=7, seed=868241
Running GSAT for n=75, instance=2, repeat=8, seed=479652
Running probSAT for n=75, instance=2, repeat=8, seed=479652
Running GSAT for n=75, instance=2, repeat=9, seed=730976



Processing n=75: 100%|██████████| 2/2 [00:04<00:00,  2.27s/it]

Running probSAT for n=75, instance=2, repeat=9, seed=730976
Running GSAT for n=75, instance=2, repeat=10, seed=50943
Running probSAT for n=75, instance=2, repeat=10, seed=50943


## Experiment 3

In [51]:

run_experiment(repeat_for_instances=10, data_num_instances_per_n=2, alg_max_tries=300000000, alg_timeout_seconds=2)

Processing instances:   0%|          | 0/8 [00:00<?, ?it/s]

Processing instance file: data/generated_instances/sat_n20_inst1.cnf



  Repeats for instance 1:  30%|███       | 3/10 [00:00<00:00, 29.04it/s]

Running GSAT for n=20, instance=1, repeat=1, seed=284616
Running probSAT for n=20, instance=1, repeat=1, seed=284616
Running GSAT for n=20, instance=1, repeat=2, seed=529116
Running probSAT for n=20, instance=1, repeat=2, seed=529116
Running GSAT for n=20, instance=1, repeat=3, seed=485057
Running probSAT for n=20, instance=1, repeat=3, seed=485057
Running GSAT for n=20, instance=1, repeat=4, seed=240498
Running probSAT for n=20, instance=1, repeat=4, seed=240498
Running GSAT for n=20, instance=1, repeat=5, seed=682357
Running probSAT for n=20, instance=1, repeat=5, seed=682357
Running GSAT for n=20, instance=1, repeat=6, seed=207299
Running probSAT for n=20, instance=1, repeat=6, seed=207299



  Repeats for instance 1:  90%|█████████ | 9/10 [00:00<00:00, 24.13it/s]

...

Running GSAT for n=75, instance=2, repeat=6, seed=476155
Running probSAT for n=75, instance=2, repeat=6, seed=476155
Running GSAT for n=75, instance=2, repeat=7, seed=919912
Running probSAT for n=75, instance=2, repeat=7, seed=919912
Running GSAT for n=75, instance=2, repeat=8, seed=661955
Running probSAT for n=75, instance=2, repeat=8, seed=661955
Running GSAT for n=75, instance=2, repeat=9, seed=978596
Running probSAT for n=75, instance=2, repeat=9, seed=978596



  Repeats for instance 2:  90%|█████████ | 9/10 [00:00<00:00, 22.54it/s]

Running GSAT for n=75, instance=2, repeat=10, seed=852502


Processing instances: 100%|██████████| 8/8 [01:23<00:00, 10.46s/it]

Running probSAT for n=75, instance=2, repeat=10, seed=852502
